In [12]:
import sklearn
import pandas as pd
import numpy as np

In [228]:
dfs = pd.read_csv('mercado2.csv', sep=';'  , engine='python', header=None)

In [243]:
dfs['qt_itens']  = len(dfs[0].str.split(",").tolist()[0])

In [245]:
for i in range(0,len(dfs)):
    dfs['qt_itens'].iloc[i] = len(dfs.iloc[i].str.split(",").tolist()[0])

/home/jp/.local/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [246]:
dfs.head()

,0,qt_itens
0,"shrimp,almonds,avocado,vegetables mix,green gr...",20
1,"burgers,meatballs,eggs",3
2,chutney,1
3,"turkey,avocado",2
4,"mineral water,milk,energy bar,whole wheat rice...",5


In [248]:
dfs.describe()

,qt_itens
count,7501.000000
mean,3.914545
std,2.905540
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,20.000000


In [16]:
from apyori import apriori

In [14]:
transactions = dfs.values.tolist()

In [46]:
dfs = dfs[0].str.split(",", expand = True)

In [48]:
dfs.fillna(0,inplace=True)

In [49]:
dfs.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,chutney,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,turkey,avocado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,mineral water,milk,energy bar,whole wheat rice,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
transactions = []
for i in range(0,len(dfs)):
    for j in range(0,20):
         if (str(dfs.values[i,j])!='0'):
            transactions.append([str(dfs.values[i,j])])

In [54]:
transactions = []
for i in range(0,len(dfs)):
    transactions.append([str(dfs.values[i,j]) for j in range(0,20) if str(dfs.values[i,j])!='0'])

In [251]:
rules = apriori(transactions, min_support = 0.01, min_confidence = 0.25)

In [252]:
#Support degree (support), first enter an empty list, and then assign
supports=[]
#Confidence
confidences=[]
#Lift
lifts=[]
#Based on items_base
bases=[]
#Derivation items items_add
adds=[]

In [253]:
for r in rules:
    for x in r.ordered_statistics:
        supports.append(r.support)
        confidences.append(x.confidence)
        lifts.append(x.lift)
        bases.append(list(x.items_base))
        adds.append(list(x.items_add))

In [254]:
df_result = pd.DataFrame({
    'support':supports,
    'confidence':confidences,
    'lift':lifts,
    'base':bases,
    'adds':adds
})

In [255]:
df_result.head()

,support,confidence,lift,base,adds
0,0.011598,0.348000,1.459926,[avocado],[mineral water]
1,0.028796,0.330275,1.837830,[burgers],[eggs]
2,0.021997,0.252294,1.476173,[burgers],[french fries]
3,0.024397,0.279817,1.173883,[burgers],[mineral water]
4,0.027463,0.338816,1.421397,[cake],[mineral water]


In [256]:
df_result = df_result[(df_result.base.str.len() >= 1) | (df_result.adds.str.len() >= 1)]

In [261]:
df_result[(df_result.support > 0.015) & (df_result.confidence > 0.28) & (df_result.lift > 1.7)].sort_values(by=['lift','confidence','support'], ascending=False).head(30)

,support,confidence,lift,base,adds
23,0.015998,0.323450,3.291994,[herb & pepper],[ground beef]
87,0.017064,0.285714,2.907928,"[mineral water, spaghetti]",[ground beef]
85,0.017064,0.416938,2.394681,"[mineral water, ground beef]",[spaghetti]
33,0.015198,0.300792,2.321232,[soup],[milk]
25,0.039195,0.398915,2.291162,[ground beef],[spaghetti]
46,0.022930,0.348178,1.999758,[olive oil],[spaghetti]
40,0.023064,0.456464,1.914955,[soup],[mineral water]
27,0.016264,0.328841,1.888695,[herb & pepper],[spaghetti]
88,0.015731,0.327778,1.882589,"[mineral water, milk]",[spaghetti]
89,0.015731,0.443609,1.861024,"[milk, spaghetti]",[mineral water]
